In [124]:
import pandas as pd 
from pycoingecko import CoinGeckoAPI

In [137]:
cg = CoinGeckoAPI()

In [127]:
def get_list_of_ecosystems(): 
    coin_cat = cg.get_coins_categories_list()
    return pd.DataFrame(coin_cat, columns = ['category_id', 'name'])

In [27]:
df_cat.to_csv('ecosystems_cat.csv', index=False)

In [150]:
import time
# fn to get list of id and name of smart contracts for a given ecosystem
def get_data_on_page(page_number, ecosystem):
    list_of_records_on_page = pd.DataFrame(cg.get_coins_markets(vs_currency='USD', category = ecosystem, page = page_number))
    try:
        return list_of_records_on_page[['id', 'name', 'last_updated']]
    except KeyError:
        return None 

coins_df = pd.DataFrame()
i = 1
while(True):
    data = get_data_on_page(i, 'ethereum-ecosystem')
    if data is not None:                
        coins_df = coins_df._append(data, ignore_index=True)

    else: 
        print("Returned Nothing on Page: "+ str(i))
        break 
    i += 1
    if len(coins_df) % 1500:
        time.sleep(5)

Returned Nothing on Page: 49


In [152]:
coins_df.to_csv('eth_coin_list.csv', index=False)

In [197]:
# def get_addresses_by_id(id):
#     # get data from page 
#     data = cg.get_coin_by_id(id)

#     # initialize dict to hold addr/s
#     platform_addresses = {}

#     for platform, address in data['platforms'].items():
#         if address.startswith('0x'):
#             platform_addresses[platform] = address

#     return platform_addresses

def get_addresses_by_id(id):
    try:
        # get data from page 
        data = cg.get_coin_by_id(id)

        # initialize dict to hold addr/s
        platform_addresses = {}

        for platform, address in data['platforms'].items():
            if address.startswith('0x'):
                platform_addresses[platform] = address

        return platform_addresses
    except ValueError:
        print("Error: ValueError occurred while retrieving data for ID:", id)
        return None


In [155]:
coins_df = coins_df[1:]

In [158]:
coins_df[98:]

,id,name,last_updated
99,safepal,SafePal,2024-02-08T09:29:33.111Z
100,worldcoin-wld,Worldcoin,2024-02-08T09:29:29.987Z
101,loopring,Loopring,2024-02-08T09:29:25.202Z
102,superfarm,SuperVerse,2024-02-08T09:29:27.310Z
103,floki,FLOKI,2024-02-08T09:29:27.682Z
...,...,...,...
4716,xave-token,Xave,2023-04-17T22:30:18.058Z
4717,yertle-the-turtle,Yertle The Turtle,None
4718,enoch,Enoch,None
4719,duel,DUEL,None


In [169]:
coins_df.where(coins_df['id'] == 'binancecoin')

,id,name,last_updated
1,NaN,NaN,NaN
2,binancecoin,BNB,2024-02-08T09:29:30.991Z
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
...,...,...,...
4716,NaN,NaN,NaN
4717,NaN,NaN,NaN
4718,NaN,NaN,NaN
4719,NaN,NaN,NaN


In [179]:
# Initialize an empty DataFrame to store the addresses
final_df = pd.DataFrame()

# Iterate over each id in the DataFrame
for _, row in coins_df.iterrows():
    id = row['id']
    print("Row: " + id)
    
    # Call the function to get platform addresses
    platform_addresses = get_addresses_by_id(id)
    
    # Convert the dictionary to a DataFrame and transpose it
    platform_df = pd.DataFrame(platform_addresses, index=[0]).transpose().reset_index()
    print(platform_df)
    
    # Rename columns
    platform_df.columns = ['platform', id]
    
    # If it's the first iteration, set the index for the final DataFrame
    if final_df.empty:
        final_df = platform_df
    else:
        # Merge the platform DataFrame with the final DataFrame on the 'platform' column
        final_df = pd.merge(final_df, platform_df, on='platform', how='outer')
    
    print("Finished row %d" % _)

# Set the 'platform' column as the index
final_df.set_index('platform', inplace=True)

Row: tether
           index                                           0
0       ethereum  0xdac17f958d2ee523a2206206994597c13d831ec7
1         fantom  0x049d68029688eabf473097a2fc38ef61633a3c7a
2  polygon-zkevm  0x1e4a5963abfd975d8c9021ce480b42188849d41d
3           kava  0x919c1c267bc06a7039e03fcc2ef738525769109c
4      avalanche  0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7
5          iotex  0x3cdb7c48e70b854ed2fa392e21687501d84b3afc
Finished row 1
Row: binancecoin
      index                                           0
0  ethereum  0xb8c77482e45f1f44de1745f52c74426c631bdd52
Finished row 2
Row: usd-coin
                       index  \
0                   ethereum   
1                     mantle   
2                     rollux   
3                      oasys   
4                thundercore   
5                  avalanche   
6              polygon-zkevm   
7                       xdai   
8        optimistic-ethereum   
9                kardiachain   
10                  neon-evm   
11  

HTTPError: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/props/

In [182]:
index = coins_df.loc[coins_df['id'] == 'props'].index[0]

In [189]:
temp_df = coins_df[1321:]

In [191]:
# Iterate over each id in the DataFrame
for _, row in temp_df.iterrows():
    id = row['id']
    print("Row: " + id)
    
    # Call the function to get platform addresses
    platform_addresses = get_addresses_by_id(id)
    
    # Convert the dictionary to a DataFrame and transpose it
    platform_df = pd.DataFrame(platform_addresses, index=[0]).transpose().reset_index()
    print(platform_df)
    
    # Rename columns
    platform_df.columns = ['platform', id]
    
    # If it's the first iteration, set the index for the final DataFrame
    if final_df.empty:
        final_df = platform_df
    else:
        # Merge the platform DataFrame with the final DataFrame on the 'platform' column
        final_df = pd.merge(final_df, platform_df, on='platform', how='outer')
    
    print("Finished row %d" % _)

# Set the 'platform' column as the index
final_df.set_index('platform', inplace=True)

Row: props
      index                                           0
0  ethereum  0x6fe56c0bcdd471359019fcbc48863d6c3e9d4f41
Finished row 1322
Row: friend-room
      index                                           0
0  ethereum  0xbde4c4a9057595fa16d38e0bd7871ef119045a84
Finished row 1323
Row: primal-b3099cd0-995a-4311-80d5-9c133153b38e
                 index                                           0
0  binance-smart-chain  0xcb5327ed4649548e0d73e70b633cdfd99af6da87
1             ethereum  0xdd13dedecebda566322195bc4451d672a148752c
2         step-network  0x0bbe45a033aab7a90006c05aa648efd3d2993f9e
Finished row 1324
Row: tribeone
                 index                                           0
0             ethereum  0xd85ad783cc94bd04196a13dc042a3054a9b52210
1  binance-smart-chain  0xd85ad783cc94bd04196a13dc042a3054a9b52210
Finished row 1325
Row: oddz
                 index                                           0
0             ethereum  0xcd2828fc4d8e8a0ede91bb38cf64b1a81de65bf6


ValueError: {'error': 'coin not found'}

In [215]:
temp_df = coins_df

In [217]:
# Iterate over each id in the DataFrame
for _, row in temp_df.iterrows():
    id = row['id']
    print("Row: " + id)
    
    # Call the function to get platform addresses
    platform_addresses = get_addresses_by_id(id)
    
    # Convert the dictionary to a DataFrame and transpose it
    platform_df = pd.DataFrame(platform_addresses, index=[0]).transpose().reset_index()
    print(platform_df)
    
    # Rename columns
    platform_df.columns = ['platform', id]
    
    # If it's the first iteration, set the index for the final DataFrame
    if final_df.empty:
        final_df = platform_df
    else:
        # Merge the platform DataFrame with the final DataFrame on the 'platform' column
        final_df = pd.merge(final_df, platform_df, on='platform', how='outer')
    
    print("Finished row %d" % _)

# Set the 'platform' column as the index
final_df.set_index('platform', inplace=True)

Row: blue-baikal
Error: ValueError occurred while retrieving data for ID: blue-baikal
Empty DataFrame
Columns: [index, 0]
Index: []
Finished row 2679
Row: pastafarian
      index                                           0
0  ethereum  0x04e3a2932bfe7b669c9f18df44b2b0a0ee681c19
Finished row 2680
Row: paw-2
      index                                           0
0  ethereum  0x419777d3e39aa9b00405724eace5ea57620c9062
Finished row 2681
Row: paw-a-gotchi
      index                                           0
0  ethereum  0x32a5fc22e20903e78171977359b500060b7316bf
Finished row 2682
Row: pawzone
      index                                           0
0  ethereum  0x1aa51bc7eb181ce48ce626bf62f8956fa9555136
Finished row 2683
Row: blocktrade-exchange
      index                                           0
0  ethereum  0x182c00807f2d4904d02d5e0d179600ff6a3ae67c
Finished row 2684
Row: blocktools
      index                                           0
0  ethereum  0xc14b4d4ca66f40f352d7a50fd230e

In [ ]:
final_df.transpose().to_csv('ethereum_smart_contracts.csv', index=False)